# MNIST·从零到一

MINIST是一个由人类手写数字构成的图像数据集。它的作者为Yann LeCun、Corinna Cortes与Christopher J.C. Burges。该数据集曾被用作分类算法的开发与评估。给定任一张图像，给出该图像对应的数字。这个任务经常被用在深度学习的入门教程中。它足够简单————包含10个类别的图像分类问题；同时又足够复杂————人类手写的数字可以非常潦草、难以辨识。

这篇教程同样以MNIST为实验内容载体。不过与其它教程不同的地方在于，我们将真正意义上从零开始，一步一步实现一个功能完善的深度学习项目。

在这个过程中你将会逐步掌握以下内容。

## 数据搜集与处理

大多数深度学习教程提供了打包好的数据集，可以实现一行代码直接调用。这种情况在现实中几乎是不可能的。有时候用于训练的数据处理工作量甚至会超过你构建模型所花费的精力。

在这个项目中，你需要自行下载MNIST数据文件，并自行编写代码实现文件读取与解析。

## 模型构建

MNIST任务相对简单，因此你不需要构建超级复杂的神经网络。在这个项目中你需要使用Keras API以两种不同的方式来构建一个深度神经网络模型。

## 模型训练

模型训练是深度神经网络成长不可获取的过程。本项目中你将会学到模型训练过程中的几个重要参数。

## 模型评估

模型评估的意义在于反映当前模型的真实水平。训练与评估是深度学习工作的一对孪生兄弟，形影不离。

## 模型保存

训练好的模型只是存活在你的内存中。你将学会如何保存自己的劳动成果，以便今后再利用。

准备好了吗？让我们开始吧！

## 认识Alex
首先介绍助教Alex，他将会伴随你的整个学习过程。

In [1]:
from tutor import Tutor
alex = Tutor("Alex")

Hi, I'm Alex. Good to see you!


## 解析MNIST

在这个章节，你需要独立完成MNIST数据集下载与文件读取功能。通常这是深度学习项目的第一步。

MNIST是一个公开数据集。你可以通过搜索引擎找到它的网址。请将你找到的网址填写在下方。


In [2]:
mnist_url = ""

### 下载文件
请将数据集文件放置在 `download` 目录下。如果文件是压缩文件，请提前将其解压缩。完成后请在下方代码块中填写对应的文件路径。

In [3]:
training_image_file = "download/train-images-idx3-ubyte"
training_label_file = "download/train-labels-idx1-ubyte"
test_image_file = "download/t10k-images-idx3-ubyte"
test_label_file = "download/t10k-labels-idx1-ubyte"

Alex将会诸葛检查你下载的文件是否完整。

In [4]:
alex.check_files(training_image_file, training_label_file, test_image_file, test_label_file)


[Alex] 训练用图像数据集文件OK!
[Alex] 训练用标签数据集文件OK!
[Alex] 测试用图像数据集文件OK!
[Alex] 测试用标签数据集文件OK!


怎么样，你下载的文件是否全部合格？

虽然现代互联网环境已经非常成熟，很少有下载文件损坏的状况出现，但是初学者入门安全第一。另外请养成从官方渠道获取原始数据的习惯。你将会在今后的工作中受益于此。

### 读取文件

使用Python读取文件是一项基础技能。神经网络的训练文件几乎全部以文件的形式存储在磁盘中，训练前的一项重要工作就是将它们读入内存，有时还需要做数据预处理与增强。不要担心，你将会在今后的项目中遇到这部分内容。今天我们先从简单的文件读取入手。

下面这段代码演示了如何从一个文本文件中读取文本内容。该文本文件仅包含3个数字，一行一个。

In [5]:
with open('text.txt', 'rt') as f:
    print(f.readline())
    print(f.readline())
    print(f.readline())

1

2

3


现在该你了。请注意，MNIST数据文件并非文本文件。你需要在它的官方文档中找到它的数据格式定义，并量身定制数据读取方法。请将你的方法实现代码填充在下方的函数中。函数的输入与输出变量已经写在 `docstring` 中。

首先是用于读取图像的函数。

In [6]:
def read_mnist_pixel(pixel_file):
    """读取MNIST数据文件中的图像像素信息。
    
    Args:
        pixel_file: 存储图像的数据集文件路径
        
    Returns:
        magic_number: magic_number
        num_images: 样本数量
        num_rows: 图像高度
        num_cols: 图像宽度
        pixels: 一个包含全部像素值的list
    """
    f = open(pixel_file, 'rb')

    magic_number = int.from_bytes(f.read(4), byteorder='big')
    num_images = int.from_bytes(f.read(4), byteorder='big')
    num_rows = int.from_bytes(f.read(4), byteorder='big')
    num_cols = int.from_bytes(f.read(4), byteorder='big')

    pixels = []
    for _ in range(num_images * num_rows * num_cols):
        pixels.append(int.from_bytes(f.read(1), byteorder='big'))
        
    f.close()

    return magic_number, num_images, num_rows, num_cols, pixels

其次是用于读取标签的函数。

In [7]:
def read_mnist_labels(label_file):
    """读取MNIST数据集文件中的标签信息。

    Args:
        label_file: 存储标签的数据集文件路径

    Returns:
        magic_number: magic_number
        num_items: 标签数量
        labels: 一个包含全部标签的list
    """
    f = open(label_file, 'rb')

    magic_number = int.from_bytes(f.read(4), byteorder='big')
    num_items = int.from_bytes(f.read(4), byteorder='big')

    labels = []
    for _ in range(num_items):
        labels.append(int.from_bytes(f.read(1), byteorder='big'))
        
    f.close()

    return magic_number, num_items, labels

Alex将会检查该函数的完成状况。

In [8]:
alex.check_reading_functions(read_mnist_pixel, read_mnist_labels)

[Alex] 正在检查图像数据集读取函数..
[Alex] 图像数据集读取方法OK！
[Alex] 正在检查标签数据集读取函数..
[Alex] 标签数据集读取方法OK！


怎么样，两个函数都满足要求了吗？你是否在 `open()` 函数的官方文档中找到了解决问题的线索？希望你不是从GitHub或者StackOverflow上复制粘贴的代码。请一定养成查阅官方文档的习惯。后期在构建神经网络时，你需要经常性地查询TensorFlow API的使用方法。